In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots
using DelimitedFiles

In [2]:
path = "./data/";

In [3]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File(string(path,"EF_SC31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File(string(path,"capacity31.csv"),header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File(string(path,"distance31.csv"),header=1,delim=",") |> DataFrame 
LCA_model = CSV.File(string(path,"LCA_model.csv"), header=1, delim=",") |> DataFrame 
SDD = CSV.File(string(path,"SDD31.csv"),header=1,delim=",") |> DataFrame;

In [4]:
global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)                          # 2021 global GD ($/yr)
es_ratio = global_sink/global_emi

emission_c = SDD[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = SDD[!, "sink ton/yr"]           # national CO2 sink (ton/yr)
Dsoc = SDD[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = collect(1:4)
cell = collect(5:10)
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"];

In [5]:
cell_demand = 500000;

---

In [12]:
function EO_SS_Model(objfunc)
    model = Model(Gurobi.Optimizer)
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) == sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15]*scaler[k]) for k in battery for j in 1:ncty]
    
    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    
    # obj func calculation
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1)  # ncty*1 matrix
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1)  # ncty*1 matrix

    transS = Vector{AffExpr}(undef, ncty)
    transD = Vector{AffExpr}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end

    Allo_soc = proD ./ emission_c .* Dsoc
    
    
    SS = sum(Allo_soc - (proD + transD))
    EO = sum(proD - proS + transD)
    
    
    if objfunc == "Ecological"
        @objective(model, Min, EO);
    else
        @objective(model, Min, SS);
    end
      
    JuMP.optimize!(model)

    opt_x = JuMP.value.(x)
    opt_y = JuMP.value.(y)
    opt_obj = JuMP.objective_value(model);
    result = Dict(["x"=>opt_x, "obj"=>opt_obj, "y"=>opt_y])
    return result
end

EO_SS_Model (generic function with 1 method)

In [13]:
function ovpct()
    model = Model(Gurobi.Optimizer)
    set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)
    
    @variable(model, x[1:ncty, 1:nproc] >= 0)
    @variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
    @variable(model, 0<=obj<=1)

    @variable(model, z[1:ncty, 1:nproc], Bin)
    @variable(model, r[1:ncty, 1:nproc], Bin)
    @variable(model, t[1:ncty, 1:nproc], Bin)
    
    x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
    x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
    x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
    x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

    cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
    cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
    cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
    cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
    cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

    @constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
    tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
    for j in tmp
        @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
    end
    
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = Matrix{AffExpr}(undef, ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
            end
            transD[i,k] = emi
        end
    end
    
    slack = 1e-5
    M = 1e7
    Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
    SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
    cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]
    
    EO = proD - proS + transD
    
    for i in 1:ncty
        for k in 1:nproc
            @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
            @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
            @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
        end
    end
    
    @constraint(model, obj*(sum(r) + sum(t)) == sum(t))
    @objective(model, Min, obj)
    JuMP.optimize!(model)
    
    xx = JuMP.value.(x)
    yy = JuMP.value.(y)
    zz = JuMP.value.(z)
    rr = JuMP.value.(r)
    tt = JuMP.value.(t)
    solution = JuMP.objective_value(model)
    
    return Dict(["x"=>xx, "obj"=>solution, "y"=>yy, "z"=>zz, "t"=>tt, "r"=>rr])
    
end 

ovpct (generic function with 1 method)

In [14]:
res_os = ovpct()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-05
Set parameter NonConvex to value 2
Set parameter IntegralityFocus to value 1
Set parameter NonConvex to value 2
Set parameter IntegralityFocus to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 2835 rows, 16276 columns and 77562 nonzeros
Model fingerprint: 0x667cb93e
Model has 1 quadratic constraint
Variable types: 14881 continuous, 1395 integer (1395 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-05, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to 

Dict{String, Any} with 6 entries:
  "obj" => 0.333333
  "t"   => [0.0 0.0 … 0.0 0.0; 1.0 0.0 … 1.0 0.0; … ; 0.0 0.0 … 8.32497e-12 0.0…
  "x"   => [0.0 0.0 … 0.0 0.0; 15479.6 0.0 … 500000.0 0.0; … ; 0.0 0.0 … 6.0396…
  "r"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…
  "z"   => [1.0 1.0 … 1.0 1.0; 0.0 1.0 … 0.0 1.0; … ; 1.0 1.0 … 1.0 1.0; 1.0 1.…
  "y"   => [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.…

In [15]:
res_eo = EO_SS_Model("Ecological")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-05
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 1409 rows, 14880 columns and 31062 nonzeros
Model fingerprint: 0x41ee94ee
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [4e-03, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 1135 rows and 13441 columns
Presolve time: 0.00s
Presolved: 274 rows, 1439 columns, 3110 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.259e+03
 Factor NZ  : 

Dict{String, Any} with 3 entries:
  "obj" => 1.05664e6
  "x"   => [6200.0 0.0 … 0.0 0.0; 5579.65 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0;…
  "y"   => [0.0 0.0 … 0.0 6200.0; 0.0 0.0 … 0.0 5579.65; … ; 0.0 0.0 … 0.0 0.0;…

---

In [21]:
function pct_count(x,y)
    proD = x .* Matrix(regional_EF[:,2:end])
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = x .* pro_sink

    transD = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            emi = 0
            for j in 1:ncty
                emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  
            end
            transD[i,k] = emi
        end
    end

    MEO = proD - proS + transD;
    
    indices = (MEO .>= 0.1) .|| (MEO .<= -0.1)
    ls = MEO[indices]
    total = length(ls)
    
    num_os = 0
    num_safe = 0
    for e in ls
        if e > 0
            num_os += 1
        elseif e < 0
            num_safe += 1
        end
    end
    
#     return proD, proS, transD

    return MEO,num_os,num_safe,total,num_os/total
end

pct_count (generic function with 1 method)

In [23]:
function net_emi(x,y)
    
    proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 
    pro_sink = zeros(ncty, nproc)
    for k in 1:nproc
        for i in 1:ncty
            pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
        end
    end
    proS = (x.*pro_sink)*ones(nproc,1) 
    
    transS = Vector{Float64}(undef, ncty)
    transD = Vector{Float64}(undef, ncty)
    for j in 1:ncty
        arc_emi = 0
        arc_seq = 0
        for i in 1:ncty
            amount = sum(y[i,j,k] for k in 1:nproc)
            arc_emi += amount * distance[!, 2:end][i,j] * EF_trans
            arc_seq += arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        end
        transD[j] = arc_emi  # ncty*1 matrix
        transS[j] = arc_seq  # ncty*1 matrix
    end
    
    EO = proD - proS + transD
    return(sum(EO))
end
    

net_emi (generic function with 1 method)

In [26]:
osx = res_os["x"]
osy = res_os["y"]
pct_opt = pct_count(osx,osy)

([0.0 0.0 … 0.0 0.0; 21624.200459696025 0.0 … 369771.2699507179 0.0; … ; 0.0 0.0 … 9.659678420069358e-6 0.0; 0.0 0.0 … 0.0 0.0], 9, 14, 23, 0.391304347826087)

In [27]:
eox = res_eo["x"]
eoy = res_eo["y"]
pct_eo = pct_count(eox,eoy)

([10278.48977681775 0.0 … 0.0 0.0; 11905.590012272154 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 974.165573281646 0.0 … 43756.84608120868 1.6167003131057103e6], 15, 15, 30, 0.5)

In [32]:
ne_os = net_emi(osx,osy)

5.04851940200074e6

In [31]:
ne_opt = net_emi(eox,eoy)

1.0566378315515348e6

---

In [129]:
model = Model(Gurobi.Optimizer)
set_optimizer_attributes(model, "NonConvex" => 2, "IntegralityFocus" => 1)

@variable(model, x[1:ncty, 1:nproc] >= 0)
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)
@variable(model, 0<=obj<=1)

@variable(model, z[1:ncty, 1:nproc], Bin)
@variable(model, r[1:ncty, 1:nproc], Bin)
@variable(model, t[1:ncty, 1:nproc], Bin)
@variable(model, beta[1:ncty, 1:nproc], Bin)

x_cth = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,5] for i in 1:ncty) * scaler[k]) for k in cathode]
x_cell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,11] for i in 1:ncty) * scaler[k]) for k in cell]
x_noncell = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,14] for i in 1:ncty) * scaler[k]) for k in noncell]
x_battery = [@constraint(model, sum(x[i,k] for i in 1:ncty) >= sum(x[i,15] for i in 1:ncty) * scaler[k]) for k in battery];

cstr_cap = [@constraint(model, x[i,k] <= capacity[!, 2:end][i,k]) for k in 1:nproc for i in 1:ncty]
cstr_op = [@constraint(model, sum(y[i,j,k] for j in 1:ncty) <= x[i,k]) for k in 1:nproc for i in 1:ncty]
cstr_cth = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,5]*scaler[k]) for k in cathode for j in 1:ncty]
cstr_cell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,11]*scaler[k]) for k in cell for j in 1:ncty]
cstr_noncell = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,14]*scaler[k]) for k in noncell for j in 1:ncty]
cstr_battery = [@constraint(model, sum(y[i,j,k] for i in 1:ncty) >= x[j,15]*scaler[k]) for k in battery for j in 1:ncty]

@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) >= cell_demand)
tmp = filter!(e->e!=mkt_loc,collect(1:ncty))
for j in tmp
    @constraint(model, sum(y[i,j,mkt_proc] for i in 1:ncty) == 0);
end

proD = x .* Matrix(regional_EF[:,2:end])
pro_sink = zeros(ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        pro_sink[i,k] = regional_EF[i, k+1] * (sink_c[i]/emission_c[i] + es_ratio)
    end
end
proS = x .* pro_sink

transD = Matrix{AffExpr}(undef, ncty, nproc)
for k in 1:nproc
    for i in 1:ncty
        emi = 0
        for j in 1:ncty
            emi += y[i,j,k] * distance[!, 2:end][i,j] * EF_trans  #PRODUCER
        end
        transD[i,k] = emi
    end
end

slack = 1e-4
M = 1e7

Allo_Dsoc = (proD*ones(nproc,1)) .* (Dsoc./emission_c)
SS = transD*ones(nproc,1) + proD*ones(nproc,1) - Allo_Dsoc
cstr_soc = [@constraint(model, SS[i] >= slack) for i in 1:ncty]

EO = proD - proS + transD

for i in 1:ncty
    for k in 1:nproc
        @constraint(model, EO[i,k] >= -M*r[i,k] + slack*t[i,k])
        @constraint(model, EO[i,k] <= -slack*r[i,k] + M*t[i,k])
        @constraint(model, z[i,k] + r[i,k] + t[i,k] == 1)
        
        @constraint(model, x[i,k] <= beta[i,k] * M)
        @constraint(model, x[i,k] >= -M * (1 - beta[i,k]) + 5)
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-05
Set parameter NonConvex to value 2
Set parameter IntegralityFocus to value 1


In [130]:
@constraint(model, obj*(sum(r) + sum(t)) == sum(t))
@objective(model, Min, obj)
JuMP.optimize!(model)

xx = JuMP.value.(x)
yy = JuMP.value.(y)
zz = JuMP.value.(z)
rr = JuMP.value.(r)
tt = JuMP.value.(t)
solution = JuMP.objective_value(model)

Set parameter NonConvex to value 2
Set parameter IntegralityFocus to value 1
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 23.10")

CPU model: 11th Gen Intel(R) Core(TM) i7-11700 @ 2.50GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3765 rows, 16741 columns and 79422 nonzeros
Model fingerprint: 0xc47013d8
Model has 1 quadratic constraint
Variable types: 14881 continuous, 1860 integer (1860 binary)
Coefficient statistics:
  Matrix range     [1e-04, 1e+07]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 1e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2708 rows and 11054 columns
Presolve time: 0.06s
Presolved: 1703 rows, 5902 columns, 26057 nonzeros
Variable types: 5649 contin

0.2600000106226735

In [121]:
pct_count(xx,yy)

([16398.174428063292 0.0 … 0.0 0.0; 82241.93995209163 0.0 … 0.0 0.0; … ; 0.0 0.0 … 9.659678420069358e-5 0.0; 0.0 0.0 … 0.0 0.0], 19, 31, 50, 0.38)

In [123]:
JuMP.value.(beta)

31×15 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0  0.0  1.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 ⋮

In [128]:
JuMP.value.(x)

31×15 Matrix{Float64}:
  6200.0       0.0          0.0  …     0.0    0.0          0.0
 61000.0       0.0          0.0        0.0    0.0          0.0
     0.0       0.0          0.0        0.0    0.0          0.0
   500.0    3100.0          0.0        0.0    9.99999      0.0
 10346.1       0.0          0.0        0.0   10.0          0.0
     0.0       0.0          0.0  …     0.0    0.0          0.0
     0.0       0.0          0.0        0.0    0.0          0.0
     0.0  145000.0          0.0        0.0    0.0          0.0
     0.0    1084.0          0.0        0.0    9.99999      0.0
     0.0       0.0          0.0        0.0   10.0          0.0
     0.0       0.0          0.0  …  2000.28   0.0          0.0
     0.0       0.0          0.0        0.0    0.0          0.0
     0.0       0.0          0.0        0.0   10.0          0.0
     ⋮                           ⋱                         
     0.0       0.0          0.0        0.0   10.0          0.0
     0.0       0.0         10.0  … 

In [117]:
EO = pct_count(xx,yy)[1]

31×15 Matrix{Float64}:
      0.0             0.0         0.0        …       0.0          0.0
      1.29143e5       0.0         0.0                0.0          0.0
      0.0             0.0         0.0                0.0          0.0
   -271.911      -65927.6         0.0               -1.45519e-11  0.0
 -56318.9             0.0         0.0               -2.08439e5    0.0
      0.0             0.0         6.73675e6  …       0.0          0.0
      0.0             0.0         0.0                0.0          0.0
      0.0             8.26326e5   0.0                0.0          0.0
      0.0            -0.0001      0.0           -75144.9          0.0
      0.0             0.0         0.0            56838.8          0.0
      0.0             0.0         0.0        …       0.0          0.0
      0.0             0.0         0.0                0.0          0.0
      0.0             0.0         0.0                0.0          0.0
      ⋮                                      ⋱                    


In [61]:
for i in 1:ncty
    for k in 1:nproc
        if EO[i,k] < 0.1 && EO[i,k] > -0.1
            EO[i,k] = 0
        end
    end
end

for i in 1:ncty
    for k in 1:nproc
        if EO[i,k] < 0
            EO[i,k] = 1
        else
            EO[i,k] = 0
        end
    end
end
EO

31×15 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮

In [64]:
diff = EO - rr

for i in 1:ncty
    for k in 1:nproc
        if diff[i,k] != 0
            println([i,k])
        end
    end
end

[21, 3]
[24, 7]
[24, 8]
[24, 12]


In [66]:
EO_ORG = pct_count(xx,yy)[1];

In [82]:
EO_ORG[21,3]*1e6

-9.999996109399945

In [68]:
EO_ORG[24,7]

0.0

In [69]:
EO_ORG[24,8]

0.0

In [75]:
EO_ORG[24,12]*1E5

-0.9999999999999999

In [71]:
EO[24,7]

0.0

In [80]:
minimum(EO_ORG)

-279740.33056179655

In [ ]:
minimium()